In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets

def circle(radius, linewidth, color):
    angles = np.linspace(0,2*np.pi,100)
    fig, ax = plt.subplots()
    ax.set_aspect(1)
    ax.set_xlim(-10,10)
    ax.set_ylim(-10,10)
    ax.plot(radius*np.cos(angles), radius*np.sin(angles), linewidth = linewidth, c = color)
    plt.title('Circle of radius {} units'.format(radius))
    plt.show()

def sine(a, linewidth, color):
    x = np.linspace(0, 2*np.pi, 100)
    fig, ax = plt.subplots()
    ax.plot(x, np.sin(a*x), linewidth = linewidth, c = color)
    plt.title('Sine wave with amplitude {} units'.format(a))
    plt.show()

interact(sine, a=widgets.FloatSlider(min=0, max=10, step=0.1, value=1), linewidth=widgets.FloatSlider(min=0, max=10, step=0.1, value=1), color=widgets.ColorPicker(value='blue'))

interactive(children=(FloatSlider(value=1.0, description='a', max=10.0), FloatSlider(value=1.0, description='l…

<function __main__.sine(a, linewidth, color)>

In [2]:
thetaReflect, reflectivity_corrected = np.genfromtxt("data.txt", unpack=True)
LAMBDA = 1.54e-10  # m

def fresnel_ideal(a_i):
    """Calculates the reflectivity of an ideal, smooth surface of Si."""
    k = 2 * np.pi / LAMBDA
    n_1 = 1
    delta_2 = 7.6e-6
    beta_2 = LAMBDA / (4 * np.pi) * 141 / 100
    n_2 = 1 - delta_2 - 1j * beta_2

    # Berechnung von k_z1 und k_z2
    k_z1 = k * np.sqrt(n_1**2 - np.square(np.cos(np.deg2rad(a_i))))
    k_z2 = k * np.sqrt(n_2**2 - np.square(np.cos(np.deg2rad(a_i))))

    # Reflexionskoeffizient
    r = (k_z1 - k_z2) / (k_z1 + k_z2)

    # Rückgabe der Reflexivität (Betrag von r zum Quadrat)
    return abs(r)**2


lambda_ka = 1.54e-10 # Angström -> Wellenlaenge der Roentgenstrahlung bei der Kalpha Linie
k = 2*np.pi/lambda_ka   # Wellenvektor
n1 = 1
d1 = 0

##### Startparameter für den Fit #####
d_poly = 1.7e-6 # 1. Schicht Polysterol
d_Sili = 8.2e-6 # 2. Schicht Silizium
b_poly = 1.8e-10
b_sili = .25e-5
d = 8.8e-8
sigma_poly = 5.2e-10
sigma_sili = 7.8e-10
######################################


theta_min = 0.2     #Startwert Fit
theta_max = 0.8     #Endwert Fit

x = np.linspace(0, 1.5, 10000)
start_params = [d_poly, d_Sili, b_poly, b_sili, d, sigma_poly, sigma_sili]  # Hier die Startparameter eintragen, die später mit *start_params übergeben werden




def parratt(alpha, delta2, delta3, b2, b3, d2, sigma1, sigma2):
    # alpha is the angle of incidence in degrees
    # delta2, delta3 are the real parts of the refractive indices of the two materials
    # b2, b3 are the imaginary parts of the refractive indices of the two materials
    # d2 is the thickness of the second layer
    # sigma1, sigma2 are the roughnesses of the interfaces    
    
    # Convert angle to radians
    alpha_rad = np.deg2rad(alpha)
    
    # Calculate refractive indices
    n2, n3 = 1.0 - delta2 - 1j * b2, 1.0 - delta3 - 1j * b3
    
    # Calculate wave vectors
    kz1 = k * np.sqrt(n1**2 - np.cos(alpha_rad)**2)
    kz2 = k * np.sqrt(n2**2 - np.cos(alpha_rad)**2)
    kz3 = k * np.sqrt(n3**2 - np.cos(alpha_rad)**2)
    
    # Calculate exponential factors for roughness
    exp_factor1 = np.exp(-2 * kz1 * kz2 * sigma1**2)
    exp_factor2 = np.exp(-2 * kz2 * kz3 * sigma2**2)
    
    # Calculate reflectivities
    r12 = ((kz1 - kz2) / (kz1 + kz2)) * exp_factor1
    r23 = ((kz2 - kz3) / (kz2 + kz3)) * exp_factor2
    
    # Calculate exponential factor for layer thickness
    exp_factor3 = np.exp(-2j * kz2 * d2)
    
    # Calculate total reflectivity
    return np.abs((r12 + exp_factor3 * r23) / (1 + r12 * exp_factor3 * r23))**2


def plot_reflectivity(d_poly, d_Sili, b_poly, b_sili, d, sigma_poly, sigma_sili, damp):
    reflectivity = parratt(thetaReflect, d_poly, d_Sili, b_poly, b_sili, d, sigma_poly, sigma_sili)
    plt.figure()
    plt.plot(thetaReflect, reflectivity_corrected/damp, label='Corrected Reflectivity')
    plt.plot(thetaReflect, reflectivity, label='Parratt Reflectivity')
    plt.xlabel('Theta (degrees)')
    plt.ylabel('Reflectivity')
    plt.legend()
    plt.yscale('log')
    plt.xlim(0,1.5)
    plt.show()

interact(plot_reflectivity, 
         d_poly=widgets.FloatSlider(min=1e-6, max=1e-5, step=1e-7, value=d_poly, readout_format='.1e'),
         d_Sili=widgets.FloatSlider(min=1e-6, max=1e-4, step=1e-7, value=d_Sili, readout_format='.1e'),
         b_poly=widgets.FloatSlider(min=1e-10, max=1e-9, step=1e-11, value=b_poly, readout_format='.1e'),
         b_sili=widgets.FloatSlider(min=1e-6, max=1e-4, step=1e-7, value=b_sili, readout_format='.1e'),
         d=widgets.FloatSlider(min=1e-8, max=1e-7, step=1e-9, value=d, readout_format='.1e'),
         sigma_poly=widgets.FloatSlider(min=1e-10, max=1e-9, step=1e-11, value=sigma_poly, readout_format='.1e'),
         sigma_sili=widgets.FloatSlider(min=1e-10, max=1e-9, step=1e-11, value=sigma_sili, readout_format='.1e'),
         damp=widgets.FloatSlider(min=1, max=10, step=0.01, value=1))

interactive(children=(FloatSlider(value=1.7e-06, description='d_poly', max=1e-05, min=1e-06, readout_format='.…

<function __main__.plot_reflectivity(d_poly, d_Sili, b_poly, b_sili, d, sigma_poly, sigma_sili, damp)>

In [5]:
interact(plot_reflectivity, 
         d_poly=widgets.FloatSlider(min=1e-6, max=1e-5, step=1e-7, value=d_poly, readout_format='.1e'),
         d_Sili=widgets.FloatSlider(min=1e-6, max=1e-4, step=1e-7, value=d_Sili, readout_format='.1e'),
         b_poly = d_poly / 200,
         b_sili = d_Sili / 400,
         d=widgets.FloatSlider(min=1e-8, max=1e-7, step=1e-9, value=d, readout_format='.1e'),
         sigma_poly=widgets.FloatSlider(min=1e-10, max=1e-9, step=1e-11, value=sigma_poly, readout_format='.1e'),
         sigma_sili=widgets.FloatSlider(min=1e-10, max=1e-9, step=1e-11, value=sigma_sili, readout_format='.1e'),
         damp=widgets.FloatSlider(min=1, max=10, step=0.01, value=1))

interactive(children=(FloatSlider(value=1.7e-06, description='d_poly', max=1e-05, min=1e-06, readout_format='.…

<function __main__.plot_reflectivity(d_poly, d_Sili, b_poly, b_sili, d, sigma_poly, sigma_sili, damp)>

In [8]:
values_before = [1.7e-6, 8.2e-6, 8.8e-8, 5.2e-10, 7.8e-10, 10]

interact(plot_reflectivity,
         d_poly=widgets.FloatSlider(min=values_before[0]-values_before[0]*.1, max=values_before[0]+values_before[0]*.1, step=1e-7, value=values_before[0], readout_format='.1e'),
         d_Sili=widgets.FloatSlider(min=values_before[1]-values_before[1]*.1, max=values_before[1]+values_before[1]*.1, step=1e-7, value=values_before[1], readout_format='.1e'),
         b_poly = d_poly / 200,
         b_sili = d_Sili / 400,
         d=widgets.FloatSlider(min=values_before[2]-values_before[2]*.1, max=values_before[2]+values_before[2]*.1, step=1e-10, value=values_before[2], readout_format='.2e'),
         sigma_poly=widgets.FloatSlider(min=values_before[3]-values_before[3]*.1, max=values_before[3]+values_before[3]*.1, step=1e-11, value=values_before[3], readout_format='.1e'),
         sigma_sili=widgets.FloatSlider(min=values_before[4]-values_before[4]*.1, max=values_before[4]+values_before[4]*.1, step=1e-11, value=values_before[4], readout_format='.1e'),
         damp=widgets.FloatSlider(min=values_before[5]-values_before[5]*.1, max=values_before[5]+values_before[5]*.1, step=0.01, value=values_before[5]))

interactive(children=(FloatSlider(value=1.7e-06, description='d_poly', max=1.87e-06, min=1.53e-06, readout_for…

<function __main__.plot_reflectivity(d_poly, d_Sili, b_poly, b_sili, d, sigma_poly, sigma_sili, damp)>